# tensorflowで深層学習
mnistの手書きデータを深層学習で分類する
基本の話
https://deepage.net/deep_learning/2016/11/07/convolutional_neural_network.html

# 毎回実行しないといけない部分
①placeholder
②scope(層の準備)
③scope(評価法の準備)
④session

## import

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 

## 画像データの読み込み

In [4]:
# MINSTの手書き画像データを読み込む --- (※1)
mnist = input_data.read_data_sets("mnist/", one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [8]:
#画像データ
#55000の画像データ
#print(mnist.train.images)

#ラベルデータ(0,1符号化されてる)
#mnist.train.labels

## 変数の用意
x : 入力変数 28×28のベクトル  
y : 出力変数(10,1)０から９

tensorboardで処理を見やすくするためにname属性で名前をつける

In [17]:
pixels = 28 * 28 # 28x28ピクセル
nums = 10 # 0-9の10クラスに分ける

# プレースホルダを定義 --- (※2)
x  = tf.placeholder(tf.float32, shape=(None, pixels), name="x") # 画像データ
y_ = tf.placeholder(tf.float32, shape=(None, nums), name="y_")  # 正解ラベル 

## 様々な関数
関数の初歩
https://qiita.com/rindai87/items/4b6f985c0583772a2e21
placeholderとvariableの違い
https://qiita.com/eve_yk/items/e42431200a1616c7d045
深層学習の関数
https://qiita.com/tadOne/items/b484ce9f973a9f80036e

tf.truncates_normal : 正規分布に従う乱数の生成
shape: 戻り値のテンソルの次元
stddev : 正規分布の標準偏差

tf.Variable : 代入可能な箱(placeholderに近い)
計算結果を保持させることを目的に利用します。計算を始める前に初期化が必要
W_init : 変数の初期値
name : tendorboardで表示される名前(別にいらん)


tf.nn.conv2d : 畳み込みを行う関数
input : 4次元([batch, in_height, in_width, in_channels])のテンソルを渡す
一番最初は画像を読み込んだ後、reshape関数で[-1, in_height, in_width, in_channels]と変換し、渡せばよい

filter(W) : 畳込みでinputテンソルとの積和に使用するweightにあたる
４次元[filter_height, filter_width, in_channels, channel_multiplier] のテンソルを渡す
最後の引数のchannel_multiplierだけchannel数が拡張される

stride : ストライド（=１画素ずつではなく、数画素ずつフィルタの適用範囲を計算するための値)を指定
ただしstrides[0] = strides[3] = 1. とする必要があるため、指定は[1, stride, stride, 1]と先頭と最後は１固定とする

padding : 「'SAME'」か「'VALID'」を利用
ゼロパディングを利用する場合はSAMEを指定

In [18]:
# 重みとバイアスを初期化する関数 --- (※3)
def weight_variable(name, shape):
    W_init = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(W_init, name="W_"+name)
    return W

def bias_variable(name, size):
    b_init = tf.constant(0.1, shape=[size])
    b = tf.Variable(b_init, name="b_"+name)
    return b

# 畳み込みを行う関数 --- (※4)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# 最大プーリングを行う関数 --- (※5)
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
        strides=[1,2,2,1], padding='SAME')

## 各層の準備
tf.name_scope()メソッド
処理をスコープに分割する
使い方
↓
with tf.name_scope('名前') as scope:
名前　：　tensorboardを使うときその工程の名前

### 畳み込み層1のデータ部分の(28,28)を使うデータの大きさで変える
### ブーリング層ではデータの大きさが毎回半分に
### 全結合レイヤーではそこでのデータの大きさによってnを変える
### 読み出し層は出力の大きさによって10の部分を変える

In [19]:
# 畳み込み層1 --- (※6)
with tf.name_scope('conv1') as scope:
    W_conv1 = weight_variable('conv1', [5, 5, 1, 32])#初期化
    b_conv1 = bias_variable('conv1', 32)#初期化
    x_image = tf.reshape(x, [-1, 28, 28, 1])#データ
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)#畳み込み(活性化関数としてReLU)

# プーリング層1 ---- (※7)
with tf.name_scope('pool1') as scope:
    h_pool1 = max_pool(h_conv1)#畳み込み層1の結果を使う

# 畳み込み層2 --- (※8)
with tf.name_scope('conv2') as scope:
    W_conv2 = weight_variable('conv2', [5, 5, 32, 64])#初期化
    b_conv2 = bias_variable('conv2', 64)#初期化
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)#プーリング層の結果を使う(活性化関数ReLU)

# プーリング層2 --- (※9)
with tf.name_scope('pool2') as scope:
    h_pool2 = max_pool(h_conv2)

# 全結合レイヤー --- (※10)
with tf.name_scope('fully_connected') as scope:
    n = 7 * 7 * 64
    W_fc = weight_variable('fc', [n, 1024])
    b_fc = bias_variable('fc', 1024)
    h_pool2_flat = tf.reshape(h_pool2, [-1, n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc) + b_fc)        

# ドロップアウト(過剰適合)を排除 --- (※11)   #全結合ではなく一部を無視(過学習対策)
with tf.name_scope('dropout') as scope:
    keep_prob = tf.placeholder(tf.float32)   #dropoutする確率を変数
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)   #全結合レイヤーを入力にして上で決めた確率でdropout
    
# 読み出し層 --- (※12)
with tf.name_scope('readout') as scope:
    W_fc2 = weight_variable('fc2', [1024, 10])#初期化
    b_fc2 = bias_variable('fc2', 10)#初期化
    y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2) + b_fc2)#softmax関数でどの数字かを0,1符号化で    

下サイトにわかりやすいイメージ図
https://lp-tech.net/articles/LVB9R

 ## モデルの学習, 評価用のスコープの設定

In [20]:
# モデルの学習 --- (※13)
with tf.name_scope('loss') as scope:
    cross_entoropy = -tf.reduce_sum(y_ * tf.log(y_conv))#交差エントロピーで損失を表す
    
with tf.name_scope('training') as scope:
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(cross_entoropy)#損失を最小にするように最小化

# モデルの評価 --- (※14)
with tf.name_scope('predict') as scope:
    predict_step = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))#予想した数字があってるか
    accuracy_step = tf.reduce_mean(tf.cast(predict_step, tf.float32))#正解率

## feed_dict
placeholderは実行時にfeed_dictという仕組みを通じて値を外挿できる仕組み
x : 画像データ
y_ : ラベルデータ
prob : dropoutの確率

In [21]:
# feed_dictの設定 --- (※15)
def set_feed(images, labels, prob):
    return {x: images, y_: labels, keep_prob: prob}

## セッションの開始
ここまでのplaceholderは
x : 画像データ
y_ : ラベルデータ
keep_prob : dropoutの確率

In [25]:
# セッションを開始 --- (※16)
with tf.Session() as sess:
    #変数の初期化(おまじない)
    sess.run(tf.global_variables_initializer())
    
    # TensorBoardの準備(おまじない)
    #このファイルのあるフォルダでtensorboard --logdir=log_dirをコマンドラインでうつと見れる
    tw = tf.summary.FileWriter("log_dir", graph=sess.graph)
    
    # テスト用のフィードを生成(フィード　 = 代入する値)
    test_fd = set_feed(mnist.test.images, mnist.test.labels, 1)
    
    # 訓練を開始 ---- (※17)
    for step in range(10):
        #ランダムに訓練データから50個選ぶ
        batch = mnist.train.next_batch(50)
        
        #batch[0] : 50個の(28,28)の画像データ
        #batch[1] :  50個の(10,1)のラベルデータ
        fd = set_feed(batch[0], batch[1], 0.5)
        
        _, loss = sess.run([train_step, cross_entoropy], feed_dict=fd)
        
        #100ステップごとに損失と正解率を出力
        if step % 2 == 0:
            acc = sess.run(accuracy_step, feed_dict=test_fd)
            print("step=", step, "loss=", loss, "acc=", acc)
            
    # 最終結果を表示
    acc = sess.run(accuracy_step, feed_dict=test_fd)
    print("正解率=", acc)


step= 0 loss= 500.696 acc= 0.1395
step= 2 loss= 400.045 acc= 0.1402
step= 4 loss= 344.247 acc= 0.1863
step= 6 loss= 293.586 acc= 0.2522
step= 8 loss= 296.805 acc= 0.2701
正解率= 0.2757


In [12]:
mnist.train.next_batch(50)[0].shape

(50, 784)